# add_refseq_genenames.ipynb
## This script takes in a refseq annotation bedfile and a set of geneID/transcript equivalencies and adds genenames to bedfile

### Import libraries

In [57]:
import math
import numpy as np
import pandas as pd

### Define refseq input and equivalency file

In [58]:
bedfile = "/Users/lysa8537/data/annotation/dm6/dm6_refseq.bed"
# Equivalency file is tab delimited, two columns, transcript then matching gene id
equivfile = "/Users/lysa8537/data/annotation/dm6/transcript_gene_equiv.txt"
basename = "dm6_refseq"

### Dump input files into variables

In [60]:
with open(bedfile) as f:
    reg = []
    for line in f:
        entry = list(line.strip().split("\t"))
        reg.append(entry)

regions = pd.DataFrame(reg)

with open(equivfile) as f:
    eq = {}
    for line in f:
        entry = list(line.strip().split("\t"))
        eq[entry[0]] = entry[1]

### Add gene names to new column of regions df

In [61]:
regions[12] = regions[3].map(eq)

### Make outfile names and write out data

In [62]:
indir = bedfile.split("/")
outdir = "/".join(indir[0:-1])
outfile = "".join([outdir, "/", basename, "_genenames_included.bed"])

regions.to_csv(outfile, sep='\t', header=False, index=False)